In [1]:
import sys
sys.path.append("..")

In [2]:
from keras.models import Model

Using TensorFlow backend.


In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [5]:
import bowizer
import tfidf
import pickle

In [6]:
with open('best_policy_random.pkl', 'rb') as f:
    gd = pickle.load(f)

In [7]:
gd[0][0]

'\n\n\n                    ________  ________  __    __  ________        \n                   |        \\|        \\|  \\  |  \\|        \\       \n                    \\$$$$$$$$| $$$$$$$$| $$  | $$ \\$$$$$$$$       \n                      | $$   | $$__     \\$$\\/  $$   | $$          \n                      | $$   | $$  \\     >$$  $$    | $$          \n                      | $$   | $$$$$    /  $$$$\\    | $$          \n                      | $$   | $$_____ |  $$ \\$$\\   | $$          \n                      | $$   | $$     \\| $$  | $$   | $$          \n                       \\$$    \\$$$$$$$$ \\$$   \\$$    \\$$          \n              __       __   ______   _______   __        _______  \n             |  \\  _  |  \\ /      \\ |       \\ |  \\      |       \\ \n             | $$ / \\ | $$|  $$$$$$\\| $$$$$$$\\| $$      | $$$$$$$\\\n             | $$/  $\\| $$| $$  | $$| $$__| $$| $$      | $$  | $$\n             | $$  $$$\\ $$| $$  | $$| $$    $$| $$      | $$  | $$\n          

In [8]:
blob ='\n\n\n                    ________  ________  __    __  ________        \n                   |        \\|        \\|  \\  |  \\|        \\       \n                    \\$$$$$$$$| $$$$$$$$| $$  | $$ \\$$$$$$$$       \n                      | $$   | $$__     \\$$\\/  $$   | $$          \n                      | $$   | $$  \\     >$$  $$    | $$          \n                      | $$   | $$$$$    /  $$$$\\    | $$          \n                      | $$   | $$_____ |  $$ \\$$\\   | $$          \n                      | $$   | $$     \\| $$  | $$   | $$          \n                       \\$$    \\$$$$$$$$ \\$$   \\$$    \\$$          \n              __       __   ______   _______   __        _______  \n             |  \\  _  |  \\ /      \\ |       \\ |  \\      |       \\ \n             | $$ / \\ | $$|  $$$$$$\\| $$$$$$$\\| $$      | $$$$$$$\\\n             | $$/  $\\| $$| $$  | $$| $$__| $$| $$      | $$  | $$\n             | $$  $$$\\ $$| $$  | $$| $$    $$| $$      | $$  | $$\n             | $$ $$\\$$\\$$| $$  | $$| $$$$$$$\\| $$      | $$  | $$\n             | $$$$  \\$$$$| $$__/ $$| $$  | $$| $$_____ | $$__/ $$\n             | $$$    \\$$$ \\$$    $$| $$  | $$| $$     \\| $$    $$\n              \\$$      \\$$  \\$$$$$$  \\$$   \\$$ \\$$$$$$$$ \\$$$$$$$ \n\n'
bloblen = len(blob)

gd[0] = [x.replace(blob, "") for x in gd[0]]

In [16]:
len_data = len(gd[1])

vocab_size = 300
training_split = 0.7

traininglen = int(len_data * training_split)

# - use gd[1] for actions data
training_set = gd[1][:traininglen]
test_set = gd[1][traininglen:]
test_set[0]

#use the game states for tokens
tm = bowizer.TokenMaker(corpus=gd[1], vocab_size= vocab_size)



In [17]:
r = np.random.RandomState(42)

In [19]:
#dont use big.txt for tokens
#with open('../../../data/norvig_tm_3000.pkl', 'rb') as f:
    #tm = pickle.load(f)

In [20]:
embed_h = EmbeddingHyper(tm.vocab_size + 1, 64)
conv_h = ConvHyper(64, 6, 4)
rnn_h = RnnHyper(512, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [21]:
dernn_h = RnnHyper(512, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(64, 6, 4)
decoder_h = td.Hyper(tm.vocab_size + 1, [dernn_h, dec_h])

In [22]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [23]:
np.array(training_set).shape

ml = 64

In [24]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [25]:
testmodel = make_model(ml)

In [26]:
#testmodel.load_weights('../models/wl_model64')

In [27]:
def training_round(history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train = bowizer.SlicedWordData(lines=r.choice(training_set, size=2000, replace=False), maxlen=ml,tokenmaker=tm)
    test = bowizer.SlicedWordData(lines=r.choice(test_set, size=400, replace=False), maxlen=ml,tokenmaker=tm)
    newhistory = testmodel.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=32,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [28]:
history = None

In [ ]:
for i in range(20):
    print(i)
    history = training_round(history)

0
Train on 2000 samples, validate on 400 samples
Epoch 1/5
2000/2000 [==============================] - 35s - loss: 1.0639 - categorical_accuracy: 0.9397 - val_loss: 0.7411 - val_categorical_accuracy: 0.9540
Epoch 2/5
2000/2000 [==============================] - 103s - loss: 0.7244 - categorical_accuracy: 0.9551 - val_loss: 0.7411 - val_categorical_accuracy: 0.9540
Epoch 3/5
2000/2000 [==============================] - 110s - loss: 0.7244 - categorical_accuracy: 0.9551 - val_loss: 0.7411 - val_categorical_accuracy: 0.9540
Epoch 4/5
2000/2000 [==============================] - 61s - loss: 0.7244 - categorical_accuracy: 0.9551 - val_loss: 0.7411 - val_categorical_accuracy: 0.9540
Epoch 5/5
2000/2000 [==============================] - 77s - loss: 0.7244 - categorical_accuracy: 0.9551 - val_loss: 0.7411 - val_categorical_accuracy: 0.9540
1
Train on 2000 samples, validate on 400 samples
Epoch 6/10
2000/2000 [==============================] - 110s - loss: 0.7397 - categorical_accuracy: 0.954

In [ ]:
testmodel.save('../models/gameaction_ae64.hp5')